In [2]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensi

In [3]:
%matplotlib inline

import os
import csv
import json
import pickle
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn.decomposition import PCA
import numpy as np
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from random import randint
random.seed(123)
np.random.seed(123)

from PIL import Image

import tensorflow as tf

In [4]:
sess = tf.InteractiveSession()

In [5]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')
df = pd.read_csv(dir + 'cleaned_reports_edited.csv')

In [6]:
exam_ids = []
for folder in os.listdir(dir+'data/Images'):
    exam_ids.append(str(folder))
    
print len(exam_ids)

330


In [7]:
sample_images = []
views = {}
for eid in exam_ids:
    for image in os.listdir(dir + 'data/Images/' + str(eid)):
        if image.endswith('.jpg'):
            item = {}
            imid = os.path.splitext(image)[0] 
            item['exam id'] = eid
            item['impath'] = eid + '/' + image
            if '_' in image and os.path.exists(dir + 'data/Images/' + str(eid) + '/' + 'SSImages_BB' + '/' + 'ss_' + image):             
                leg = imid.split('_')[-1]
                
                if leg != 'B':
                    view = imid.split('_')[-2]

                    item['imid'] = imid #.split('_')[0]
                    item['leg'] = leg
                    item['view'] = view
                    if view not in views.keys():
                        views[view] = 0
                    views[view] = views[view] + 1

                    f_ = open(dir + 'data/Images/' + str(eid) + '/' + 'SSImages_BB' + '/'  + image + '.txt', 'r')
                    item['bbox'] = f_.readlines()[1].rstrip('\n').split(' ')
                else:
                    continue
            
            else:
                continue


            sample_images.append(item)
print views
print len(sample_images)

{'HBL': 12, 'WBAP': 25, 'L': 90, 'AP': 61, 'S': 26, 'WBL': 17}
231


In [8]:
train, test = model_selection.train_test_split(sample_images, train_size=0.8, random_state=42)

print len(train), len(test)

184 47


/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
# Crop and pad images, adjust bounding box coordinates

def crop_image(im, bbox):
    # Central crop to 256x256
    im = np.array(im)
    h, w, _ = im.shape
    #print 'Before: ',h,w
    if h > 256 and w > 256:
        im = im[h//2-128:h//2+128, w//2-128:w//2+128]
        bbox[0] = bbox[0] - (w-256)//2
        bbox[1] = bbox[1] - (h-256)//2
        bbox[2] = bbox[2] - (w-256)//2
        bbox[3] = bbox[3] - (h-256)//2
    elif h > 256 and w < 256:
        #print 'width<256 ', p
        p = abs(w-256)//2
        im = im[h//2-128:h//2+128, :]
        if abs(w-256)%2==0:
            im = np.pad(im, ((0,0), (p,p), (0,0)), 'maximum')
        else:
            im = np.pad(im, ((0,0), (p,p+1), (0,0)), 'maximum')
        bbox[0] = bbox[0] + p
        bbox[1] = bbox[1] - (h-256)//2
        bbox[2] = bbox[2] + p
        bbox[3] = bbox[3] - (h-256)//2
    elif h < 256 and w > 256:
        p = abs(h-256)//2
        #print 'height<256 ', p
        im = im[:,  w//2-128:w//2+128]
        if abs(h-256)%2==0:
            im = np.pad(im, ((p,p), (0,0), (0,0)), 'maximum')
        else:
            im = np.pad(im, ((p,p+1), (0,0), (0,0)), 'maximum')
        bbox[0] = bbox[0] - (w-256)//2
        bbox[1] = bbox[1] + p
        bbox[2] = bbox[2] - (w-256)//2
        bbox[3] = bbox[3] + p
    elif h < 256 and w < 256:
        ph = abs(h-256)//2
        pw = abs(w-256)//2
        print pw, ph
        im = np.pad(im, ((ph,ph), (pw,pw), (0,0)), 'maximum')
        bbox[0] = bbox[0] + pw
        bbox[1] = bbox[1] + ph
        bbox[2] = bbox[2] + pw
        bbox[3] = bbox[3] + ph
    
    # if bounding boxes fall out of range, set to max height/width
    if bbox[0] < 0:
        bbox[0] = 0
    if bbox[1] < 0:
        bbox[1] = 0
    if bbox[2] > 256:
        bbox[2] = 256
    if bbox[3] > 256:
        bbox[3] = 256
    #rawim = np.copy(im).astype('uint8')
    h, w, _ = im.shape
    #print 'After: ', h,w
    return im, bbox

In [10]:
def next_batch(data, batch_size=100, display_images = False):
    batch = random.sample(data, batch_size)
    images = []
    bboxes = []
    for i in batch:
        if display_images:
            plt.figure()
        
        path = dir + 'data/Images/' + str(i['exam id']) + '/' + 'SSImages_BB' + '/'  + 'ss_' + i['imid'] + '.jpg'
        im = plt.imread(path)
        bbox = np.asarray(i['bbox'], dtype=int)
        if display_images:
        #print 'Original: ', im.shape, 'Original bbox: ', i['bbox']
            plt.subplot(1,2,1).set_title('Org. bbox: ' + str(bbox))
            plt.subplot(1,2,1).imshow(im)
            rect1 = patches.Rectangle((bbox[0],bbox[1]),abs(bbox[2]-bbox[0]),abs(bbox[3]-bbox[1]),linewidth=1,edgecolor='r',facecolor='none')
            plt.subplot(1,2,1).add_patch(rect1)
        
        im, bbox = crop_image(im, bbox)
        if display_images:
        #print 'Cropped: ', im.shape, 'Adjusted bbox: ', bbox
            plt.subplot(1,2,2).set_title('Adj. bbox: ' + str(bbox))
            plt.subplot(1,2,2).imshow(im)
            rect2 = patches.Rectangle((bbox[0],bbox[1]),abs(bbox[2]-bbox[0]),abs(bbox[3]-bbox[1]),linewidth=1,edgecolor='r',facecolor='none')
            plt.subplot(1,2,2).add_patch(rect2)
        
        images.append(im)
        bboxes.append(bbox)

    return np.array(images), np.array(bboxes)

In [11]:
images, bboxes = next_batch(train, batch_size=184)
print images.shape

(184, 256, 256, 3)


## Linear regression model

In [19]:
x = tf.placeholder(tf.float32, shape=[None, 256*256*3])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

W = tf.Variable(tf.zeros([256*256*3,4]))
b = tf.Variable(tf.zeros([4]))


sess.run(tf.global_variables_initializer())

y = tf.matmul(x,W) + b

loss = tf.reduce_mean(tf.losses.mean_squared_error(y_, y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [20]:
BS = 20
print_y = tf.Print(y,[y])

for i in range(1000):
    images, bboxes = next_batch(train, batch_size=20)
    images = np.reshape(images, [BS, 256*256*3])
    train_step.run(feed_dict={x: images, y_: bboxes})
    if i%100==0:
        print loss.eval(feed_dict={x: images, y_: bboxes})

1.27476e+18
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

## CNN regression model

In [ ]:
# CNN
# Input Layer
input_layer = tf.placeholder(tf.float32, [None, 256, 256, 3])

# Convolutional Layer #1
conv1 = tf.layers.conv2d(
    inputs=input_layer,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# Pooling Layer #1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(
    inputs=dense, rate=0.4, training=tf.estimator.ModeKeys.TRAIN == tf.estimator.ModeKeys.TRAIN)

# Output Layer
output = tf.layers.dense(inputs=dropout, units=4)

# Loss
loss = tf.reduce_mean(tf.losses.mean_squared_error(bboxes, output))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
      for i in range(20000):
        images, bboxes = next_batch(train, batch_size=10)
        if i % 100 == 0:
            train_loss = loss.eval(feed_dict={
                x: images, y_: bboxes})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
